In [1]:
# from PyQt5.QtWidgets import QApplication, QWidget, QLabel,QPushButton,QLineEdit,QMessageBox, QScrollArea, QVBoxLayout,QRadioButton,QListWidget,QListWidgetItem ,QHBoxLayout, QFormLayout, QGroupBox
from PyQt5.QtGui import QIcon, QPixmap
from PyQt5 import QtCore
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
import sys
import requests
# import time
# import pandas as pd
# import datetime
from bs4 import BeautifulSoup
# from openpyxl import load_workbook
import os
import re
import urllib
import threading
from functools import partial
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

In [2]:
# myLabel 製作Qlabel可以被點集發送訊號
class myLabel(QLabel):
    clicked = pyqtSignal()
    def mouseReleaseEvent(self, QMouseEvent):
        if QMouseEvent.button() == Qt.LeftButton:
            self.clicked.emit()

class App(QWidget):
    
    def __init__(self):
        super().__init__()
        self.title = 'MangaDownloader'
        self.left = 10
        self.top = 10
        self.width = 600
        self.height = 800
        self.count = 1
        self.pageSize = 10
        
        self.data_dict = self.Manga_info(self.count,self.pageSize)
        self.my_list = QListWidget()
        self.whole_layout = QVBoxLayout()
        self.mangaInfo = QVBoxLayout()
        self.headers = QHBoxLayout()
        self.search = QHBoxLayout()
        self.manga_layout = QHBoxLayout()
        self.bt_layout = QHBoxLayout()
        self.setLayout(self.whole_layout)
        self.initUI()

    def initUI(self):
        self.setWindowTitle(self.title)
        
        
        header_textbox1 = QLabel("點選想要查看的漫畫圖片")
        header_textbox2 = QLabel("點選想要下載的漫畫集數")
        header_textbox3 = QPushButton("查看下載的漫畫")
        self.headers.addWidget(header_textbox1,3)
        self.headers.addWidget(header_textbox2,3)
        self.headers.addWidget(header_textbox3,1)
        self.header_widget = QWidget()
        self.header_widget.setLayout(self.headers)
        self.whole_layout.addWidget(self.header_widget)
        
        self.search_le = QLineEdit()
#         self.le.setObjectName("host")
        self.search_le.setText("")
        self.search_le.setPlaceholderText("輸入想要查找的漫畫關鍵字")

        self.search_bt = QPushButton()
        self.search_bt.setText("搜尋") 
        self.search_bt.clicked.connect(self.serach_button_click)
        self.search.addWidget(self.search_le)
        self.search.addWidget(self.search_bt)
        self.search_widget = QWidget()
        self.search_widget.setLayout(self.search)
        self.whole_layout.addWidget(self.search_widget)
        
        #取得漫畫資訊
#         data_dict = self.Manga_info(self.count,self.pageSize)
        
#         #取得漫畫集數
#         chapter = self.Manga_chapterList(data_dict['UpdateComicItems'][0]['UrlKey'])
        
        formLayout = QFormLayout()
        self.groupBox = QGroupBox()
        
        for manga in self.data_dict['UpdateComicItems']:
        # Create widget 製作漫畫圖框
            label = myLabel()
            while(1):
                if (os.path.exists('./photo/%s.jpg'%str(manga['ID']))):
                    label.setPixmap(QPixmap(str('./photo/%s.jpg'%str(manga['ID']))))
                    break
            label.setFixedSize(200,300)
            
            label.clicked.connect(partial(self.myLabel_clicked,manga))
            # Create widget 製作漫畫名稱框
            textbox = QLabel(manga['Title'])
            label.show()
            formLayout.addRow(textbox, label)
        self.groupBox.setLayout(formLayout)

        self.scroll = QScrollArea()
        self.scroll.setWidget(self.groupBox)
        self.scroll.setWidgetResizable(True)
        
        self.manga_layout.addWidget(self.scroll)

#         self.mangaInfo.addWidget(self.label)
#         self.mangaInfo.addWidget(self.textbox)
#         self.my_widget_1 = QWidget()
#         self.my_widget_1.setLayout(self.mangaInfo)
        
        #將漫畫資訊 漫畫有幾集 傳入create_chapter_list 添加至layout內
#         self.create_chapter_list(data_dict,chapter)
        
        self.my_widget = QWidget()
        self.my_widget.setLayout(self.manga_layout)
        
        self.whole_layout.addWidget(self.my_widget)

        self.create_pre_nex_bt()
        self.resize(800, 600)
        
        self.show()
    
    def create_pre_nex_bt(self):
        try:
            self.my_widget_2.close()
        except:
            pass
        self.page_text = QLabel(str(self.count))
        self.page_text.setParent(self)
        self.page_text.setAlignment(QtCore.Qt.AlignCenter | QtCore.Qt.AlignVCenter)

        self.prev_button = QPushButton('<')
        self.prev_button.setParent(self)
        self.next_button = QPushButton('>')
        self.next_button.setParent(self)
        self.prev_button.clicked.connect(self.preb_button_click)
        self.next_button.clicked.connect(self.next_button_click)
        self.bt_layout.addWidget(self.prev_button)
        self.bt_layout.addWidget(self.page_text)
        self.bt_layout.addWidget(self.next_button)
        self.my_widget_2 = QWidget()
        self.my_widget_2.setLayout(self.bt_layout)
        self.whole_layout.addWidget(self.my_widget_2)
    
    def create_chapter_list(self,data_dict,chapter):
        try:
            self.my_list.close()
        except:
            pass
        self.my_list = QListWidget()
        self.my_list.setParent(self)
#         self.my_list.move(300,100)
        for i,key in zip(range(len(chapter)),chapter):
            download_dict = {"manga_src":chapter[key][0],
                             "manga_name":data_dict['Title'],
                             "manga_pic":data_dict['ShowPicUrlB'],
                             "manga_chap":key}
            #新建个Item
            item = QListWidgetItem()
            #将item添加到list
            self.my_list.addItem(item)
            
            button = QPushButton(key)
#             button.setFixedSize(50,100)
            button.clicked.connect(partial(self.onClicked,download_dict))
            self.my_list.setItemWidget(item,button)
#         self.my_list.setFixedSize(200,100)
        self.manga_layout.addWidget(self.my_list)
        self.my_list.show()
    
    def search_manga_info(self):
        Web_Site_URL = "https://www.manhuaren.com/pagerdata.ashx?"
        user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
        headers = { 'User-Agent' : user_agent }
        form_data = {
            't':'7',
            'pageindex':self.count,
            'f':'0',
            'title':self.search_le.text()}
        data_dict = requests.post(Web_Site_URL,headers=headers,data=form_data,verify = False).json()
        new_data_dict = {}
        for i in data_dict:
            
            if('UpdateComicItems' in new_data_dict):
                new_data_dict['UpdateComicItems'].append({'ID':i['Id'],'Title':i['Title'],'UrlKey':i['Url'],'ShowPicUrlB':i['BigPic'],})
            else:
                new_data_dict['UpdateComicItems']= []
                new_data_dict['UpdateComicItems'].append({'ID':i['Id'],'Title':i['Title'],'UrlKey':i['Url'],'ShowPicUrlB':i['BigPic'],})
        for manga in new_data_dict['UpdateComicItems']:
            threading.Thread(target = self.Manga_make_jpg,args = (manga,)).start()
        return new_data_dict

    def serach_button_click(self):
        self.groupBox.close()
        data_dict = self.search_manga_info()
        
        #取得漫畫集數
#         chapter = self.Manga_chapterList(data_dict['UpdateComicItems'][0]['UrlKey'])
        
        formLayout = QFormLayout()
        self.groupBox = QGroupBox()
        
        for manga in data_dict['UpdateComicItems']:
        # Create widget 製作漫畫圖框
            label = myLabel()
            while(1):
                if (os.path.exists('./photo/%s.jpg'%str(manga['ID']))):
                    label.setPixmap(QPixmap(str('./photo/%s.jpg'%str(manga['ID']))))
                    break
#             pixmap = QPixmap(str('./photo/%s.jpg'%str(manga['ID'])))
            label.setFixedSize(200,300)
#             label.setPixmap(pixmap)
            label.clicked.connect(partial(self.myLabel_clicked,manga))
            # Create widget 製作漫畫名稱框
            textbox = QLabel(manga['Title'])
            formLayout.addRow(textbox, label)
        self.groupBox.setLayout(formLayout)
#         self.groupBox.show()
#         self.scroll = QScrollArea()
        self.scroll.setWidget(self.groupBox)
#         self.scroll.setWidgetResizable(True)
        self.groupBox.show()

    def next_button_click(self):
#         self.create_pre_nex_bt()
        self.count = self.count+1
        self.groupBox.close()
        if(self.search_le.text() == ''):
            #取得漫畫資訊
            print('取得一般漫畫')
            data_dict = self.Manga_info(self.count,self.pageSize)
        else:
            print('取得搜尋漫畫')
            data_dict = self.search_manga_info()
            
        #取得漫畫集數
#         chapter = self.Manga_chapterList(data_dict['UpdateComicItems'][0]['UrlKey'])
        
        formLayout = QFormLayout()
        self.groupBox = QGroupBox()
        
        for manga in data_dict['UpdateComicItems']:
        # Create widget 製作漫畫圖框
            label = myLabel()
            while(1):
                if (os.path.exists('./photo/%s.jpg'%str(manga['ID']))):
                    label.setPixmap(QPixmap(str('./photo/%s.jpg'%str(manga['ID']))))
                    break
            label.setFixedSize(200,300)
#             label.setPixmap(pixmap)
            label.clicked.connect(partial(self.myLabel_clicked,manga))
            # Create widget 製作漫畫名稱框
            textbox = QLabel(manga['Title'])
            formLayout.addRow(textbox, label)
        self.groupBox.setLayout(formLayout)
#         self.groupBox.show()
#         self.scroll = QScrollArea()
        self.scroll.setWidget(self.groupBox)
#         self.scroll.setWidgetResizable(True)
        self.groupBox.show()
        
#         data_dict = self.Manga_info(self.count,self.pageSize)

#         chapter = self.Manga_chapterList(data_dict['UpdateComicItems'][0]['UrlKey'])
#         self.create_chapter_list(data_dict,chapter)

#         pixmap = QPixmap(str('%s.jpg'%str(self.count)))
#         self.label.setPixmap(pixmap)
        self.page_text.setText(str(self.count))
#         self.textbox.setText(data_dict['UpdateComicItems'][0]['Title'])

    def preb_button_click(self):
#         self.create_pre_nex_bt()
        if(self.count>1):
            self.count = self.count-1
            self.groupBox.close()
            
            if(self.search_le.text() == ''):
                #取得漫畫資訊
                print('取得一般漫畫')
                data_dict = self.Manga_info(self.count,self.pageSize)
            else:
                print('取得搜尋漫畫')
                data_dict = self.search_manga_info()

#             #取得漫畫資訊
#             data_dict = self.Manga_info(self.count,self.pageSize)

            #取得漫畫集數
#             chapter = self.Manga_chapterList(data_dict['UpdateComicItems'][0]['UrlKey'])

            formLayout = QFormLayout()
            self.groupBox = QGroupBox()

            for manga in data_dict['UpdateComicItems']:
            # Create widget 製作漫畫圖框
                label = myLabel()
                while(1):
                    if (os.path.exists('./photo/%s.jpg'%str(manga['ID']))):
                        label.setPixmap(QPixmap(str('./photo/%s.jpg'%str(manga['ID']))))
                        break
                label.setFixedSize(200,300)
#                 label.setPixmap(pixmap)
                label.clicked.connect(partial(self.myLabel_clicked,manga))
                # Create widget 製作漫畫名稱框
                textbox = QLabel(manga['Title'])
                formLayout.addRow(textbox, label)
            self.groupBox.setLayout(formLayout)
    #         self.groupBox.show()
    #         self.scroll = QScrollArea()
            self.scroll.setWidget(self.groupBox)
    #         self.scroll.setWidgetResizable(True)
            self.groupBox.show()
#             data_dict = self.Manga_info(self.count)
#             chapter = self.Manga_chapterList(data_dict['UpdateComicItems'][0]['UrlKey'])
#             self.create_chapter_list(data_dict,chapter)

#             pixmap = QPixmap(str('%s.jpg'%str(self.count)))
#             self.label.setPixmap(pixmap)
            self.page_text.setText(str(self.count))
#             self.textbox.setText(data_dict['UpdateComicItems'][0]['Title'])
        else:
            buttonReply = QMessageBox.question(self, '提醒', "已經在最前面了", QMessageBox.Ok, QMessageBox.Ok)
            if buttonReply == QMessageBox.Ok:
                print('Yes clicked.')
    
    def myLabel_clicked(self,manga):
        #取得漫畫集數
        chapter = self.Manga_chapterList(manga['UrlKey'])
        self.create_chapter_list(manga,chapter)
        
    def Manga_info(self,pageindex,pagesize):
        Web_Site_URL = "http://www.manhuaren.com/manhua-list-area36-s2/dm5.ashx?"
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
        headers = { 'User-Agent' : user_agent }
        form_data = {
            'action':'getclasscomics',
            'pageindex':pageindex,
            'pagesize':pagesize,
            'categoryid':0,
            'tagid':0,
            'status':0,
            'usergroup':0,
            'pay':-1,
            'areaid':36,
            'sort':2,
            'iscopyright':0}

        data_dict = requests.post(Web_Site_URL,headers=headers,data=form_data,verify = False).json()
        for manga in data_dict['UpdateComicItems']:
            threading.Thread(target = self.Manga_make_jpg,args = (manga,)).start()
        return data_dict

    def Manga_make_jpg(self,manga):
        img_url = manga['ShowPicUrlB']
        try:
            os.mkdir('./photo')
        except:
            pass
        if (os.path.exists('./photo/%s.jpg'%str(manga['ID']))):
            pass
        else:
            res = requests.get(img_url,verify = False).content
            with open('./photo/%s.jpg'%str(manga['ID']),'wb') as f:     #二進位制寫入
                f.write(res)
            
    def Manga_chapterList(self,manga_UrlKey):
#         manga_index = 'http://www.manhuaren.com/%s/?from=/manhua-list-area36-s2/'%manga_UrlKey
        manga_index = 'http://www.manhuaren.com/%s/'%manga_UrlKey
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
        headers = { 'User-Agent' : user_agent }
        index = requests.get(manga_index,headers=headers, verify = False)
        soup = BeautifulSoup(index.text,"lxml")
        chapterRawLists = soup.find_all('a',class_='chapteritem')
        chapter_list = {}
        for i in chapterRawLists:
            chapter_list[i.text] = i.get_attribute_list('href')
        return chapter_list
    
    def onClicked(self,download_dict):
        button = self.sender()
        button.close()
        threading.Thread(target = self.download_manga,args = (download_dict,)).start()
        
    def download_manga(self,download_dict):
        print(download_dict)
        manga_src = download_dict['manga_src']
        manga_name = download_dict['manga_name']
        img_url = download_dict['manga_pic']
        manga_chap = download_dict['manga_chap']
        manga_index = ('https://www.manhuaren.com/%s/'%(manga_src))
        user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
        headers = { 'User-Agent' : user_agent }
        index = requests.get(manga_index,headers=headers)
        soup = BeautifulSoup(index.text,"lxml")
        Episode = soup.find_all('script')
        image_URL_part = str(Episode[-3]).split("'")[-4].split("|")
        try:
            os.mkdir(("./manga"))
        except Exception as e:
            pass
        try:
            os.mkdir(("./manga/%s/")%(manga_name))
            if (os.path.exists('./manga/%s/index.jpg'%(manga_name))):
                pass
            else:
                res = requests.get(img_url,verify = False).content
                with open('./manga/%s/%s.jpg'%(manga_name,'index'),'wb') as f:     #二進位制寫入
                    f.write(res)
        except Exception as e:
            pass
        pages = []
        for pt in image_URL_part:
            if (pt.find("_")!= (-1)):
                pages.append(pt)

        url_base = re.split('\.|/|-|://|\?|=|&',str(Episode[-3]).split("'")[-7])
        print(url_base)
        print(image_URL_part)
        # a = 10
        url_sample = []
        for i in range(len(url_base)):
            if( i == 11):
                url_sample.append("Pages_number")
                continue
            if(url_base[i].isdigit()):
#                 if((i)==5):
#                     url_sample.append(url_base[i])
#                 else:
                url_sample.append(image_URL_part[int(url_base[i])])
            elif(url_base[i].islower()):
                url_sample.append(image_URL_part[int((ord(url_base[i])-87))])
            else:
                url_sample.append(url_base[i][0])
        print(url_sample)
        for i in pages:
            url_sample[11] = i
            url = ('%s://%s-%s-%s-%s-%s.%s.%s/%s/%s/%s/%s.%s?%s=%s&%s=%s&%s=%s')%tuple(url_sample)
            print(url)
#             self.download_manga_page(url,manga_name,manga_chap,str(i),img_tp)
            
            try:
                
                try:
                    os.mkdir(("./manga/%s/%s/")%(manga_name,manga_chap))
                except Exception as e:
                    pass
#                     print(e)
                if (os.path.exists('./manga/%s/%s/%s.%s'%(manga_name,manga_chap,str(i),url_sample[12]))):
                    continue
                else:
                    res = requests.get(url,verify = False).content
                    with open('./manga/%s/%s/%s.%s'%(manga_name,manga_chap,str(i),url_sample[12]),'wb') as f:     #二進位制寫入
                        f.write(res)
            except Exception as e:
                print(e)
                continue
            
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
#     app.exec_()
    sys.exit(app.exec_())


取得一般漫畫


SystemExit: 0

c:\python37\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
Web_Site_URL = "http://www.manhuaren.com/manhua-list-area36-s2/dm5.ashx?"
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers = { 'User-Agent' : user_agent }
form_data = {
    'action':'getclasscomics',
    'pageindex':1,
    'pagesize':1,
    'categoryid':0,
    'tagid':0,
    'status':0,
    'usergroup':0,
    'pay':-1,
    'areaid':36,
    'sort':2,
    'iscopyright':0}

data_dict = requests.post(Web_Site_URL,headers=headers,data=form_data,verify = False).json()
print(data_dict)

{'UpdateComicItems': [{'ID': 55820, 'Title': '被反常规的英雄养育大的、常识外的魔法剑士', 'UrlKey': 'manhua-beifanchangguideyingxiongyangyudade-changshiwaidemofajianshi', 'Logo': '', 'LastPartUrl': 'm1037675', 'ShowLastPartName': '第6话 ', 'ShowPicUrlB': 'http://mhfm8us.cdnmanhua.net/56/55820/20191213183343_180x240_18.jpg', 'ShowConver': 'http://mhfm4us.cdnmanhua.net/56/55820/20191213183422_480x369_43.jpg', 'ComicPart': '被反常规的英雄养育大的、常识外的魔法剑士 第6话', 'Author': ['雪月佳', 'kt60 '], 'ShowReads': '1892.1万', 'Content': '常识外的魔法剑士漫画 ，为了救少女而丧命的少年，转生后过...', 'Star': 3, 'ShowSource': None, 'Status': 0, 'LastUpdateTime': '8分钟前更新', 'ShelvesTime': '2019-12-13 发布'}], 'Count': 23285}


In [ ]:
data_dict

In [ ]:
img_url = data_dict['UpdateComicItems'][0]['ShowPicUrlB']

# 弄圖片
# res = requests.get(img_url).content
# with open('美女.jpg','wb') as f:     #二進位制寫入
#     f.write(res)

In [6]:
# manga_index = 'https://www.manhuaren.com/%s/?from=/manhua-list-area36-s2/'%data_dict['UpdateComicItems'][0]['UrlKey']
manga_index = 'https://www.manhuaren.com/m1036548/'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers = { 'User-Agent' : user_agent }
index = requests.get(manga_index,headers=headers)
soup = BeautifulSoup(index.text,"lxml")
Episode = soup.find_all('script')

In [33]:
Episode = soup.find_all('a',class_='chapteritem')
# print(Episode)
# print(Episode.get_attribute_list('href'))
data_dict = {}
for i in Episode:
    data_dict[i.text] = i.get_attribute_list('href')
print(data_dict)

{}


In [39]:
for i in data_dict:
    print(i)
    print(data_dict[i][0])

In [46]:
# print(str(Episode[-3]).split("'"))
for i in str(Episode[-3]).split("'"):
    if ("|") in i :
        print(i)

.replace(/^/,String)){while(c--)d[e(c)]=k[c]||e(c);k=[function(e){return d[e]}];e=function(){return
1036548||cdnmanhua|219|42|net|250|manhua1034|https|139|2aca0c184c5ae287a077cfd505d7da03|key|104|type|41644|jpg|cid|4_5848|5_1968|6_6975|3_8315|newImgs|1_3807|2_4934|7_3713|12_8102|var|13_6040|11_8039|8_4298|9_2104|10_9801
|


In [47]:
m1036548 = str(Episode[-3]).split("'")[-4]
print(m1036548)

1036548||cdnmanhua|219|42|net|250|manhua1034|https|139|2aca0c184c5ae287a077cfd505d7da03|key|104|type|41644|jpg|cid|4_5848|5_1968|6_6975|3_8315|newImgs|1_3807|2_4934|7_3713|12_8102|var|13_6040|11_8039|8_4298|9_2104|10_9801


In [44]:
m1032707= str(Episode[-3]).split("'")[-4]
print(m1032707)

1032707||cdnmanhua|219|42|net|104|manhua1034|https|250|c3eaafdadb65aebaa081e5688bdd8c76|key|139|type|41644|cid|jpg|4_6571|3_4703|2_4223|1_5983|newImgs|5_2455|var|9_6972|8_2950|6_5977|7_7549


In [41]:
m1036643 = str(Episode[-3]).split("'")[-4]
print(m1036643)

1036643||cdnmanhua|139|62|net|104|manhua1034|https|250|e70bbb5fda5a2b74c80821f76abbef65|key|219|type|cid|61636|jpg|3_5550|2_3851|1_6965|newImgs|4_5680|var|7_1786|5_5709|6_1096


In [37]:
for i,URL_pt in zip(range(len(str(Episode[-3]).split("'")[-4].split('|'))),str(Episode[-3]).split("'")[-4].split('|')):
    print("index:",i)
    print(URL_pt)

index: 0
1036643
index: 1

index: 2
cdnmanhua
index: 3
139
index: 4
62
index: 5
net
index: 6
104
index: 7
manhua1034
index: 8
https
index: 9
250
index: 10
e70bbb5fda5a2b74c80821f76abbef65
index: 11
key
index: 12
219
index: 13
type
index: 14
cid
index: 15
61636
index: 16
jpg
index: 17
3_5550
index: 18
2_3851
index: 19
1_6965
index: 20
newImgs
index: 21
4_5680
index: 22
var
index: 23
7_1786
index: 24
5_5709
index: 25
6_1096


In [31]:
for i,URL_pt in zip(range(len(str(Episode[-3]).split("'")[-4].split('|'))),str(Episode[-3]).split("'")[-4].split('|')):
    print("index:",i)
    print(URL_pt)

index: 0
1032707
index: 1

index: 2
cdnmanhua
index: 3
219
index: 4
42
index: 5
net
index: 6
104
index: 7
manhua1034
index: 8
https
index: 9
250
index: 10
c3eaafdadb65aebaa081e5688bdd8c76
index: 11
key
index: 12
139
index: 13
type
index: 14
41644
index: 15
cid
index: 16
jpg
index: 17
4_6571
index: 18
3_4703
index: 19
2_4223
index: 20
1_5983
index: 21
newImgs
index: 22
5_2455
index: 23
var
index: 24
9_6972
index: 25
8_2950
index: 26
6_5977
index: 27
7_7549


In [63]:
# print(max(len(m1036548.split('|')),len(m1032707.split('|')),len(m1036643.split('|'))))
for i in range(max(len(m1036548),len(m1032707),len(m1036643))):
    print(i)
    print(m1032707.split('|')[i])
    print(m1032707.split('|')[i].isdigit())
    
    print(m1036548.split('|')[i],m1032707.split('|')[i],m1036643.split('|')[i])
    print()

0
1032707
True
1036548 1032707 1036643

1

False
  

2
cdnmanhua
False
cdnmanhua cdnmanhua cdnmanhua

3
219
True
219 219 139

4
42
True
42 42 62

5
net
False
net net net

6
104
True
250 104 104

7
manhua1034
False
manhua1034 manhua1034 manhua1034

8
https
False
https https https

9
250
True
139 250 250

10
c3eaafdadb65aebaa081e5688bdd8c76
False
2aca0c184c5ae287a077cfd505d7da03 c3eaafdadb65aebaa081e5688bdd8c76 e70bbb5fda5a2b74c80821f76abbef65

11
key
False
key key key

12
139
True
104 139 219

13
type
False
type type type

14
41644
True
41644 41644 cid

15
cid
False
jpg cid 61636

16
jpg
False
cid jpg jpg

17
4_6571
False
4_5848 4_6571 3_5550

18
3_4703
False
5_1968 3_4703 2_3851

19
2_4223
False
6_6975 2_4223 1_6965

20
1_5983
False
3_8315 1_5983 newImgs

21
newImgs
False
newImgs newImgs 4_5680

22
5_2455
False
1_3807 5_2455 var

23
var
False
2_4934 var 7_1786

24
9_6972
False
7_3713 9_6972 5_5709

25
8_2950
False
12_8102 8_2950 6_1096

26
6_5977
False


IndexError: list index out of range

In [61]:
url = '%s://%s-%s-%s-%s-%s.%s.%s/%s/%s/%s/'%(m1032707.split('|')[8],
                                    m1032707.split('|')[7],
                                    m1032707.split('|')[6],
                                    m1032707.split('|')[9],
                                    m1032707.split('|')[12],
                                    m1032707.split('|')[3],
                                    m1032707.split('|')[2],
                                    m1032707.split('|')[5],
                                    m1032707.split('|')[4],
                                    m1032707.split('|')[14],
                                    m1032707.split('|')[0],)
print(url)
# https://manhua1034-104-250-139-219.cdnmanhua.net/42/41644/1032707/5_2455.jpg?cid=1032707&type=1

https://manhua1034-104-250-139-219.cdnmanhua.net/42/41644/1032707


In [54]:
print(m1032707.split('|'))

['1032707', '', 'cdnmanhua', '219', '42', 'net', '104', 'manhua1034', 'https', '250', 'c3eaafdadb65aebaa081e5688bdd8c76', 'key', '139', 'type', '41644', 'cid', 'jpg', '4_6571', '3_4703', '2_4223', '1_5983', 'newImgs', '5_2455', 'var', '9_6972', '8_2950', '6_5977', '7_7549']


In [15]:
manga_src = 'm1036695'
# manga_src = 'm1036643'
# manga_src = 'm1036548'

manga_index = ('https://www.manhuaren.com/%s/'%(manga_src))
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers = { 'User-Agent' : user_agent }
index = requests.get(manga_index,headers=headers)
soup = BeautifulSoup(index.text,"lxml")
Episode = soup.find_all('script')

image_URL_part = str(Episode[-3]).split("'")[-4].split("|")
pages = []
Numbers = []
for pt in image_URL_part[:14]:
#     print(pt)
    if(pt.isdigit()):
        Numbers.append(pt)
    elif (pt.find('manhua')!= (-1)):
        first_domain = pt
    else:
        continue
print(first_domain)
print("**********")
for pt in image_URL_part[14:]:
    print(pt)
    if(pt == 'cid'):
        continue
    elif (pt.find("_")!= (-1)):
#         print("頁碼：",pt) 1_xxx
        pages.append(pt)
    elif (pt.isdigit()):
#         print("sid:",pt) 41644
        sid = pt
    elif (pt == 'newImgs'):
        continue
    elif (pt == 'var'):
        continue
    else:
        img_tp = pt #jpg/png/...
        print("unknow:",pt)
for i in pages:
    url = '%s://%s-%s-%s-%s-%s.%s.%s/%s/%s/%s/%s.%s?cid=%s&type=1'%('https',)
    print(url)
#     res = requests.get(url,verify = True).content
#     with open('%s.jpg'%str(i),'wb') as f:     #二進位制寫入
#         f.write(res)
#https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/6_1094.jpg?cid=1036695&type=1
# https://manhua1034-104-250-139-219.cdnmanhua.net/42/41644/1032707/5_2455.jpg?cid=1032707&type=1

manhua1034
**********
61646
jpg
unknow: jpg
cid
6_1094
5_1290
8_6005
7_1775
4_3816
1_3570
newImgs
3_5755
2_1212
15_8684
14_6463
16_3013
var
13_9087
10_4495
9_3774
12_9828
11_1023
https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/6_1094.jpg?cid=1036695&type=1
https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/5_1290.jpg?cid=1036695&type=1
https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/8_6005.jpg?cid=1036695&type=1
https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/7_1775.jpg?cid=1036695&type=1
https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/4_3816.jpg?cid=1036695&type=1
https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/1_3570.jpg?cid=1036695&type=1
https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/3_5755.jpg?cid=1036695&type=1
https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/2_1212.jpg?cid=1036695&type=1
https://manhua1034-104-250-139-219.cdnmanhua.net/62/6

In [13]:
Numbers

['1036695', '219', '62', '250', '104', '139']

In [3]:
#https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/6_1094.jpg?cid=1036695&type=1
imgurl = "https://manhua1034-104-250-139-219.cdnmanhua.net/42/41644/1032707/5_2455.jpg?cid=1032707&type=1"
# os.mkdir(os.path.dirname("./%s/%s/"%("海盗高达dust",'第26話')))

In [27]:
if not os.path.exists(os.path.dirname('./学妹前世是你妈/第16话')):
    os.mkdir(('./学妹前世是你妈/'))
    os.mkdir(('./学妹前世是你妈/第16话/'))
# os.path.exists(os.path.dirname('./学妹前世是你妈/第16话/'))

In [5]:
import urllib

In [7]:
opener=urllib.request.build_opener()


opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)

urllib.request.urlretrieve(imgurl,'pic.jpg')


('pic.jpg', <http.client.HTTPMessage at 0x11447b290>)

In [2]:
manga_index = ('https://www.manhuaren.com/m1036911/')
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
headers = { 'User-Agent' : user_agent }
index = requests.get(manga_index,headers=headers)
soup = BeautifulSoup(index.text,"lxml")
Episode = soup.find_all('script')
# print(index.text)
image_URL_part = str(Episode[-3]).split("'")[-4].split("|")
print(image_URL_part)

['1036911', '', '16', 'cdnmanhua', 'net', 'manhua1034', 'https', '250', '104', '15558', '150', '74fee7e878ff04f2dd2afade7389ee3d', 'type', 'key', 'jpg', 'cid', '16_4081', '17_7147', '15_5561', '13_4442', '14_6631', '12_1125', '23_1020', '21_8531', '22_2379', '18_6974', '19_7670', '20_1150', '3_9984', '4_3569', '5_9004', 'newImgs', '1_3382', '2_6395', '9_3642', '10_6873', '11_4974', '6_7191', '7_2753', '8_3389', '24_3174', '40_3182', '41_6738', '39_5940', '37_9336', '38_3247', '42_9798', '46_2226', '31_5099', '45_5565', '43_1275', '44_4378', '36_6048', '28_5527', '29_5192', '27_1639', '25_2243', '26_5306', '30_3831', '34_8756', '35_1363', '33_3303', 'var', '32_8727', '']


In [30]:
url_base = re.split('\.|/|-|://|\?|=|&',str(Episode[-3]).split("'")[-7])
print(url_base)
# a = 10
url_sample = []
for i in range(len(url_base)):
    if( i == 11):
        url_sample.append("Pages_number")
        continue
    if(url_base[i].isdigit()):
        if((i)==5):
            url_sample.append(url_base[i])
        else:
            url_sample.append(image_URL_part[int(url_base[i])])
    elif(url_base[i].islower()):
        url_sample.append(image_URL_part[int((ord(url_base[i])-87))])
    else:
        url_sample.append(url_base[i][0])
print(url_sample)
# print(re.split('\.|/',url_sample))
# page_url = re.split('\.|/',url_sample)
# del page_url[1]
# print(page_url)
print(('%s://%s-%s-%s-%s-%s.%s.%s/%s/%s/%s/%s.%s?%s=%s&%s=%s&%s=%s')%tuple(url_sample))

['6', '5', '8', '7', 'a', '12', '3', '4', '2', '9', '0', 'L', 'e', 'f', '0', 'd', 'b', 'c', '1\\']
['https', 'manhua1034', '104', '250', '150', '12', 'cdnmanhua', 'net', '16', '15558', '1036911', 'Pages_number', 'jpg', 'cid', '1036911', 'key', '877f189c0e8be38a04770e3bd15d8c2f', 'type', '1']
https://manhua1034-104-250-150-12.cdnmanhua.net/16/15558/1036911/Pages_number.jpg?cid=1036911&key=877f189c0e8be38a04770e3bd15d8c2f&type=1


In [31]:
# https://manhua1034-104-250-139-219.cdnmanhua.net/62/61646/1036695/6_1094.jpg?cid=1036695&type=1
# 0://1-2-3-4-5.6.7/8/9/10/11.12?13=14&15=16
url_sample

['https',
 'manhua1034',
 '104',
 '250',
 '150',
 '12',
 'cdnmanhua',
 'net',
 '16',
 '15558',
 '1036911',
 'Pages_number',
 'jpg',
 'cid',
 '1036911',
 'key',
 '877f189c0e8be38a04770e3bd15d8c2f',
 'type',
 '1']

In [32]:
url_sample[11] = '1_1234'
print(url_sample)

['https', 'manhua1034', '104', '250', '150', '12', 'cdnmanhua', 'net', '16', '15558', '1036911', '1_1234', 'jpg', 'cid', '1036911', 'key', '877f189c0e8be38a04770e3bd15d8c2f', 'type', '1']


In [4]:
def Manga_info(pageindex,pagesize):
    Web_Site_URL = "http://www.manhuaren.com/manhua-list-area36-s2/dm5.ashx?"
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
    headers = { 'User-Agent' : user_agent }
    form_data = {
        'action':'getclasscomics',
        'pageindex':pageindex,
        'pagesize':pagesize,
        'categoryid':0,
        'tagid':0,
        'status':0,
        'usergroup':0,
        'pay':-1,
        'areaid':36,
        'sort':2,
        'iscopyright':0}

    data_dict = requests.post(Web_Site_URL,headers=headers,data=form_data,verify = False).json()
#     threading.Thread(target = Manga_make_jpg,args = (data_dict,)).start()
    return data_dict
data_dict = Manga_info(1,10)

In [5]:
data_dict

{'UpdateComicItems': [{'ID': 59174,
   'Title': 'AREA51',
   'UrlKey': 'manhua-area-51',
   'Logo': '',
   'LastPartUrl': 'm1037447',
   'ShowLastPartName': '第48话 ',
   'ShowPicUrlB': 'http://mhfm2us.cdnmanhua.net/60/59174/20200601111802_180x240_36.jpg',
   'ShowConver': 'http://mhfm1us.cdnmanhua.net/60/59174/20200601111832_480x369_99.jpg',
   'ComicPart': 'AREA51 第48话',
   'Author': ['久正人'],
   'ShowReads': '2140.3万',
   'Content': '美国传说中的第51区，其中聚集着各种非人类生物。偶尔也有...',
   'Star': 3,
   'ShowSource': None,
   'Status': 0,
   'LastUpdateTime': '12分钟前更新',
   'ShelvesTime': '04月20号 发布'},
  {'ID': 16476,
   'Title': 'Urara 迷路帖',
   'UrlKey': 'manhua-urara-milutie',
   'Logo': '',
   'LastPartUrl': 'm1037440',
   'ShowLastPartName': '第53话 ',
   'ShowPicUrlB': 'http://mhfm8us.cdnmanhua.net/17/16476/20190325130816_180x240_24.jpg',
   'ShowConver': 'http://mhfm8us.cdnmanhua.net/17/16476/20190325130742_480x369_58.jpg',
   'ComicPart': 'Urara 迷路帖 第53话',
   'Author': ['はリかも'],
   'ShowReads': '2896.

In [7]:
def Manga_info(pageindex,pagesize):
    Web_Site_URL = "http://www.manhuaren.com/manhua-list-area36-s2/dm5.ashx?"
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'
    headers = { 'User-Agent' : user_agent }
    form_data = {
        'action':'getclasscomics',
        'pageindex':pageindex,
        'pagesize':pagesize,
        'categoryid':0,
        'tagid':0,
        'status':0,
        'usergroup':0,
        'pay':-1,
        'areaid':36,
        'sort':2,
        'iscopyright':0}

    data_dict = requests.post(Web_Site_URL,headers=headers,data=form_data,verify = False).json()
    for manga in data_dict['UpdateComicItems']:
        threading.Thread(target = Manga_make_jpg,args = (manga,)).start()
    return data_dict

def Manga_make_jpg(manga):
    img_url = manga['ShowPicUrlB']
    res = requests.get(img_url,verify = False).content
    try:
        os.mkdir('./photo')
    except:
        pass
    with open('./photo/%s.jpg'%str(manga['ID']),'wb') as f:     #二進位制寫入
        f.write(res)
data_dict = Manga_info(1,10)

In [7]:
Web_Site_URL = "https://www.manhuaren.com/pagerdata.ashx?"
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
headers = { 'User-Agent' : user_agent }
# form_data = {
#     't':7,
#     'pageindex':1,
#     'f':0,
#     'title':'租借'}
form_data = {'t':int(7), 'pageindex':int(1),'f':int(0), 'title':'租借'}

data_dict = requests.post(Web_Site_URL,headers=headers,data=form_data,verify = False).json()
print(data_dict)



[{'Id': 41178, 'Author': ['时田时雨 '], 'Url': '/manhua-zujienvyouyuetianxiaojie/', 'Pic': 'https://mhfm1us.cdnmanhua.net/42/41178/20180927192621_320x246_39.jpg', 'Title': '租借女友月田小姐', 'LastUpdateInfo': '第14话', 'LastPartShowName': '第14话', 'Mid': 41178, 'ReadUrl': '', 'BigPic': 'https://mhfm7us.cdnmanhua.net/42/41178/20180313115940_180x240_23.jpg', 'Content': '26岁社畜星野理人，上一次与女性好好说话是什么时候呢？为了想要能够与女性好好交流而选择了低难度的约会！那就是租借女友！——然而出现在会面场所的这个奇妙的女性是……？', 'TagList': ['搞笑'], 'LastPartUrl': 'm823885', 'AuthorUrl': ['/search?title=%e6%97%b6%e7%94%b0%e6%97%b6%e9%9b%a8+&language=1&f=2'], 'LastPartTime': '2019-05-21', 'AddTime': '2018-03-13', 'Categorys': '', 'Score': None, 'Status': '连载中'}, {'Id': 583, 'Author': ['PEACH-PIT'], 'Url': '/manhua-jiangshijiedai/', 'Pic': 'https://mhfm7us.cdnmanhua.net/1/583/20160826164720_320x246_31.jpg', 'Title': '僵尸借贷', 'LastUpdateInfo': '第13卷', 'LastPartShowName': '第13卷', 'Mid': 583, 'ReadUrl': '', 'BigPic': 'https://mhfm5us.cdnmanhua.net/1/583/583.jpg', 'Content': '纪多满是个「心中有